# Info bici

In [1]:
import json
import pickle
from shapely import MultiLineString, LineString, Point

import geopandas as gpd
import osmnx as ox
import networkx as nx

from my_paths import *
import Graph_functions

## Strade/Ciclabili

### Piste ciclabili

Prima fase di pulizia del geojson raw

In [2]:
gdf_ciclabili_raw = gpd.read_file(PATH_CICLABILI_RAW)
gdf_ciclabili_staging = gdf_ciclabili_raw.copy()

# Convertiamo alcuni tipi
gdf_ciclabili_staging["geometry"] = gdf_ciclabili_staging["geometry"].apply(lambda x: x.geoms[0])
gdf_ciclabili_staging["fine_lavor"] = gdf_ciclabili_staging["fine_lavor"].astype(int)
gdf_ciclabili_staging["lunghezza"] = gdf_ciclabili_staging["lunghezza"].astype(int)
gdf_ciclabili_staging = gdf_ciclabili_staging.drop(["id_amat", "id_via","gerarchia"], axis=1)

gdf_ciclabili_staging = gpd.GeoDataFrame(gdf_ciclabili_staging, geometry="geometry")

Creazione del grafo + archi di congiunzione tra nodi molto vicini (5 metri).
Aggiunta di alcuni attributi utili:
- **"tipo"** = Per identificare in futuro la provenienza degli archi.
- **"artificiale"** = "False" se riferiti a veri percorsi presi dal geojson, "True" se aggiunti per scopi di calcolo percorsi.
N.B: "geojson_to_graph()" manterrà all'interno degli archi del grafo anche tutti gli attributi delle colonne del gdf.

In [3]:
G_ciclabili = Graph_functions.geojson_to_graph(gdf_ciclabili_staging, weight_moltiplicator=0, tipo = "Ciclabile", artificiale = False).to_undirected()
G_ciclabili = Graph_functions.add_edge_near_nodes(G_ciclabili, weight_moltiplicator = 0, tipo = "Ciclabile", artificiale = True)

Salviamo grafo (in pickle) e geojson in staging

In [4]:
gdf_edges = ox.graph_to_gdfs(G_ciclabili, edges=True, nodes=False)
gdf_edges.to_file(PATH_CICLABILI_GEOJSON_STAGING, driver="GeoJSON") # Anche questo andrebbe ripulito
with open(PATH_CICLABILI_PICKLE_STAGING, "wb") as f:
    pickle.dump(G_ciclabili, f)

### Strade ciclabili

#### Ingestion (openstreet maps)

In [5]:
place = "Milano, Italy"

bike_filter = (
    '["highway"]["bicycle"!~"no"]["bicycle"!~"dismount"]'
)
G_ox_strade = ox.graph_from_place(place, network_type='bike', custom_filter=bike_filter)
with open(PATH_STRADE_CICLABILI_PICKLE_RAW, "wb") as f:
    pickle.dump(G_ox_strade, f)

#### ETL 

1. Scelta attributi da mantenere
2. Impostare pesi (weight) in base a: Lunghezza strada e tipo di strada
3. Aggiunti di attributi custom, come:
    - **"artificiale"** = Di base "False" ma sarà "True" con aggiunta di archi ad hoc a scopi di calcolo percorsi minimi
    - **"geometry"** = Manteniamo il dato originale della geometria all'interno degli archi. Utile per conversione in gdf.
    - **"tipo"** = Attributo per distinguere la provenienza degli archi. Utile quando verranno uniti in un solo grafo.
4. Aggiunta di archi artificiali per connettere nodi molto vicini non connessi.

In [6]:
with open(PATH_STRADE_CICLABILI_PICKLE_RAW, "rb") as f:
    G_strade = pickle.load(f)

for u, v, k, data in G_strade.edges(data=True, keys=True):
    # Scelgo gli attributi da mantenere
    new_data = {}
    new_data['length'] = data['length']
    new_data['name'] = data.get('name', None)
    new_data['weight'] = data['length'] * 1 # Per ora
    new_data["tipo"] = "Strade_ciclabili"
    new_data["artificiale"] = False
    new_data["geometry"] = data.get("geometry", LineString([(G_strade.nodes[u]["x"], G_strade.nodes[u]["y"]),
                                        (G_strade.nodes[v]["x"], G_strade.nodes[v]["y"])]))
    data.clear()
    data.update(new_data)

G_strade = Graph_functions.add_edge_near_nodes(G_strade.to_undirected(), # undirected -> Doppio senso
                                               weight_moltiplicator=1,  # Per ora
                                               tipo = "Strade_ciclabili", artificiale = True)

Salvataggio in staging, sia grafo che geojson.

In [7]:
gdf_edges = ox.graph_to_gdfs(G_strade, edges=True, nodes=False)

gdf_edges.to_file(PATH_STRADE_CICLABILI_GEOJSON_STAGING, driver="GeoJSON")

with open(PATH_STRADE_CICLABILI_PICKLE_STAGING, "wb") as f:
    pickle.dump(G_strade, f)

### Unione Ciclabili e Strade

Load da staging i 2 Grafi:
- Piste **Ciclabili**
- **Strade** in cui è permesso usare la bici

In [8]:
# Carichiamo Graph Ciclabili
with open(PATH_CICLABILI_PICKLE_STAGING, "rb") as f:
    G_ciclabili = pickle.load(f)

# Carichiamo Graph Strade
with open(PATH_STRADE_CICLABILI_PICKLE_STAGING, "rb") as f:
    G_strade = pickle.load(f)

1. Unione dei due Grafi tramite: **nx.compose()**.
2. Aggiunta di archi artificiali per unire i nodi dei 2 Grafi, in modo che possano essere "comunicanti"  
(si possa passare da un arco "strade" a un arco "ciclabili" e viceversa)

In [9]:
G_compose = nx.compose(G_strade, G_ciclabili)
G_compose_merged = Graph_functions.add_edge_near_nodes(G_compose,
                                                       weight_moltiplicator = 0,
                                                       tipo = "Join_ciclabile_strade",
                                                       artificiale=True)

In [ ]:
with open(PATH_RETE_CICLABILE_COMPLETA_PICKLE_CLEAN, "wb") as f:
    pickle.dump(G_strade, f)

gdf_compose = ox.graph_to_gdfs(G_compose_merged, edges=True, nodes=False)
gdf_compose.to_file(PATH_RETE_CICLABILE_COMPLETA_GEOJSON_CLEAN, driver="GeoJSON")

## Stazioni BikeMi

In [11]:
gdf = gpd.read_file(PATH_BIKEMI_RAW, sep = ';')
gdf = gdf.rename(columns = {'id_amat':'id_stazione',
                           'stalli':'n_posti',
                           'LONG_X_4326':'longitudine',
                           'LAT_Y_4326': 'latitudine',
                           'Location':'location'})
gdf.drop(['stato', 'id_via', 'indirizzo', 'civico', 'zd_attuale', 'anno', "tipo"], axis=1, inplace=True)
gdf['icon'] = "share"
gdf["geometry"] = gdf.apply(lambda row: Point(row['longitudine'], row['latitudine']), axis=1)
gdf = gdf.set_geometry("geometry", crs=4326)
gdf = gdf.drop(columns="location")

c:\Matteo\Coding\Project\Progetto-Bike\.venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver CSV does not support open option SEP
  return ogr_read(


In [12]:
gdf.to_file(PATH_BIKEMI_CLEAN, driver="GeoJSON")